# Intro
I chose the city of Tempe, AZ as the subject of this project, for sevreal reasons, I have had lived there for 5 years, therfore I have a good knowlege of the area. Intially, I wanted to do on my hometown, Doha, but after I downloaded the raw data it looked that the data has not been updated in a while, also I have noticed that I will have a hard time with the Arabic letters in the data, so I resorted to do the project on Tempe.

# Exploring the nature of the data
**Note: the template of the code was provided by udacity quizzes**



In [15]:
# import packages
import csv, codecs, pprint, re, cerberus, operator
import xml.etree.cElementTree as ET

file_path = 'Phoenix'

def get_element(file_path):
    
    context = ET.iterparse(file_path, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end': 
            yield elem
            root.clear
    

# Count tags
def count_tags(file_in):
    """
    Fill out the count_tags function. It should return a dictionary with the 
    tag name as the key and number of times this tag can be encountered in 
    the map as value.
    """
    
    tags = {}
    
    for event, element in ET.iterparse(file_in):
        if element.tag in tags:
            tags[element.tag] += 1
            
        else:
            tags[element.tag] = 1
    

    return tags
        
print count_tags(file_path)



{'node': 318952, 'member': 27051, 'nd': 407790, 'tag': 279359, 'bounds': 1, 'note': 1, 'meta': 1, 'relation': 837, 'way': 51849, 'osm': 1}


Below I will use Regex to find how the data is formatted.
3 patters are provided: <Enter> 

1- "lower", for tags that contain only lowercase letters and are valid. <Enter> 

2- "lower_colon", for otherwise valid tags with a colon in their names.

3- "problemchars", for tags with problematic characters


In [8]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
       
        # use REGEX for lower
        if lower.search(element.attrib['k']):
            keys['lower'] += 1
            
        # Use Regex for lower_colon
        elif lower_colon.search(element.attrib['k']) :
            keys['lower_colon'] += 1
            
        # Use Regex for Problem Characters
        elif problemchars.search(element.attrib['k']):
            keys['problemchars'] += 1
            
        else:
            keys['other'] += 1
        
        pass
        
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

process_map(file_path)

{'lower': 187453, 'lower_colon': 88074, 'other': 3831, 'problemchars': 1}

Below I will discover the most contributed users (by username) for the Phoenix metropolitan street map.

In [11]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint
import re


def get_user(element):
    return element.get('user')

count = 0

def process_map(filename):
    
    user_count = {}
    for _, element in ET.iterparse(filename):
        
        user_name = get_user(element)
        
        
        
        
        
        
        if user_name in user_count:
            user_count[user_name] += 1
        else:
            user_count[user_name] = 1
        
        
        
       
        
        
        
        
    

    return user_count

user_count = process_map(file_path)


print user_count


{'pluton_od': 135, 'pinkythepig': 2, 'Sundance': 3002, 'Brian@Brea': 55, 'ashleyannmathew': 10, 'srsjojo': 65, 'OSMF Redaction Account': 11, 'robinsonsenterprises': 1, 'woowoo256': 14, 'RichRico': 1228, 'xybot': 5, 'Sarr_Cat': 89, 'ramyaragupathy': 216, 'cgu66': 3, 'Joe2oh': 11, 'menuderiaguanajuato': 1, 'Manu1400': 1, 'Meercatnip': 1, 'The Mapster': 66, 'yurasi': 843, 'Stormy Thomas': 1, 'Stephen214': 88, 'Ollie': 95, 'Pnrrth': 14, 'JayHen': 2, 'maxerickson': 263, 'skquinn': 3, '25or6to4': 45, 'j03lar50n': 1, 'sivan00': 5, 'pilotrobert': 6, 'azstumbler': 2, 'wd1933c': 4, 'osmjwh': 25, 'Your Village Maps': 2031, 'ArizonaMapper': 431, 'me0': 2, 'Marco | Wimmer PLLC': 1, 'saikabhi': 72, 'odna': 3, 'Kadou88': 1, 'bondah': 1, 'BuganiniQ': 1, 'fidcastro': 2, 'Minh Nguyen': 2, 'LeTopographeFou': 1001, 'Margter': 1, 'FvGordon': 16, 'Travi5': 23, u'\u042e\u043a\u0430\u0442\u0430\u043d': 30582, 'Christy Allen': 10, 'oldtopos': 20, 'thesuntheory': 48, 'woodpeck_repair': 6, 'Spanholz': 536, 'kgha

In [14]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
           'Ave': 'Avenue',
           'Blvd.' : 'Boulevard',
           'Pkwy' : 'Parkway',
           'Rd.' : 'Road',
           'Rd' : 'Road',
           'Dr' : 'Drive',
           'Rd,' : 'Road'
          
           
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):
    

    if '#' in name:
        name = name.split('#',-1)[0].strip()
        
    if ',' in name:
        name = name.split(',', 1)[0].strip()
        
        
    
    
    
    
    
        
    m = street_type_re.search(name)
    
   

    
    
    if m:
        street_type = m.group()
        if street_type not in expected:
            if street_type in mapping:
    
                name= re.sub(street_type_re, mapping[street_type] , name)
        
        




    return name


def test():
    st_types = audit(OSMFILE)
    assert len(st_types) == 3
    pprint.pprint(dict(st_types))

    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name
            if name == "West Lexington St.":
                assert better_name == "West Lexington Street"
            if name == "Baldwin Rd.":
                assert better_name == "Baldwin Road"








# the problem here is the pattern is also picking up non street names such as #111
st_types = audit(file_path)

for st_type, ways in st_types.iteritems():
    for name in ways:
        print update_name(name, mapping)

NameError: global name 'defaultdict' is not defined

# Fixing and shaping the data into CSV files
**Note: the template of the code was provided by udacity quiz**

In [13]:
#!/usr/bin/env python
# -*- coding: utf-8 


import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = 'Phoenix'

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    
    way_attribs = {}
    way_nodes_dict = {}
    
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
    way_tag_dict = {}
    

    # YOUR CODE HERE
    
    
    if element.tag == 'node':
        for node in NODE_FIELDS:
            node_attribs[node] = element.attrib[node]
        for child in element:
            tag_dict = {}
            
            if child.tag == 'tag':
                tag_dict['id'] = element.attrib['id'] # set the id = parent id
                
                
                
                # and child.attrib['v'] here
                tag_dict['value'] = child.attrib['v']
                
                n = re.search(PROBLEMCHARS, child.attrib['k'] )
                m = re.search(LOWER_COLON, child.attrib['k'])
                
                if n:
                    # you access child.attrib['k']
                    tag_dict['key'] = child.attrib['k']
                    continue
                    
                    
                elif m:
                    # in case of colon in key, clean to standardize the data
                    tag_dict['key'] = child.attrib['k'].split(':')[0]
                    tag_dict['type'] = child.attrib['k'].split(':',1)[0]
                    
                
                else:
                    # if it does not meet the above criteria, assign it as it is
                    tag_dict['key'] = child.attrib['k']
                    tag_dict['type'] = 'Regular'
                
                #print tag_dict works fine here
                # append the result of dictionaries to tags list
                # Note it is not appending as it supposed to
                tags.append(tag_dict)
        
            

        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        # implement way_attribs here
        for way in WAY_FIELDS:
            way_attribs[way] = element.attrib[way]
            
            
            
        # implement childern in elements here (way_nodes_dict)
        counter = 0
        for child in element:
            if child.tag == 'tag':
                tag = {'id': way_attribs['id']}
                k = child.get('k')
                if not PROBLEMCHARS.search(k):
                    k = k.split(':', 1)
                    tag['key'] = k[-1]
                    tag['value'] = child.get('v')
                    if len(k) == 1:
                        tag['type'] = 'regular'    
                    elif len(k) == 2:
                        tag['type'] = k[0]
                tags.append(tag)
                
                
            if child.tag == 'nd':
                nd = {'id' : way_attribs['id']}
                nd['node_id'] = child.get('ref')
                nd['position'] = counter
                way_nodes.append(nd)
            counter += 1
                
                
        
            
        
            
                
        
                
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'wb') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'wb') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'wb') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'wb') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'wb') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])



process_map(OSM_PATH, validate=False)




# Converting CSV files into SQLite Database

In [3]:
import sqlite3, csv
from pprint import pprint


sqlite_file = 'mydb_NEW.db' # name of db file

# connect to database:
conn = sqlite3.connect(sqlite_file)
conn.text_factory = str

# Get a cursor object 
cur = conn.cursor() # Return a cursor for the connection.

'''nodes_tags section'''

cur.execute('''DROP TABLE IF EXISTS nodes_tags''') # exucutes an sql statement
conn.commit()


# Create the table, specifying the column names and data types:
cur.execute('''
    CREATE TABLE nodes_tags(id INTEGER, key TEXT, value TEXT,type TEXT)
''')
# commit the changes
conn.commit()


# open nodes_tags.csv
with open('nodes_tags.csv', 'rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]
    

    


# inseart the formatted data
cur.executemany(
    'INSERT INTO nodes_tags(id, key, value,type) VALUES (?, ?, ?, ?);', to_db)
    
conn.commit()







''' nodes section'''

cur.execute('''DROP TABLE IF EXISTS nodes''') # exucutes an sql statement
conn.commit()


cur.execute('''
    CREATE TABLE nodes(id INTEGER, lat DECIMAL, lon DECIMAL, user TEXT, uid INTEGER, version INTEGER , changeset INTEGER, timestamp TIMESTAMP)
''')
# commit the changes
conn.commit()
# Read the csv file as a dictionary, 
# then format as a list of tuples

# open nodes.csv
# id	lat	lon	user	uid	version	changeset	timestamp
with open('nodes.csv', 'rb') as fin:
    dr = csv.DictReader(fin)
    to_db = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]
    
cur.executemany('INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?,?,?);', to_db)





''' ways section'''

cur.execute('''DROP TABLE IF EXISTS ways''') # exucutes an sql statement
conn.commit()

#id	user	uid	version	changeset	timestamp


cur.execute('''
    CREATE TABLE ways(id INTEGER, user TEXT, uid INTEGER, version INTEGER , changeset INTEGER, timestamp TIMESTAMP)
''')
# commit the changes
conn.commit()
# Read the csv file as a dictionary, 
# then format as a list of tuples

# open ways.csv
with open('ways.csv', 'rb') as fin:
    dr = csv.DictReader(fin)
    to_db = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]
    
cur.executemany('INSERT INTO ways(id, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?);', to_db)




''' ways nodes section'''

cur.execute('''DROP TABLE IF EXISTS ways_nodes''') # exucutes an sql statement
conn.commit()

#id	node_id	position


cur.execute('''
    CREATE TABLE ways_nodes(id INTEGER, node_id INTEGER, position INTEGER)
''')
# commit the changes
conn.commit()
# Read the csv file as a dictionary, 
# then format as a list of tuples

# open ways_nodes.csv
with open('ways_nodes.csv', 'rb') as fin:
    dr = csv.DictReader(fin)
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]
    
cur.executemany('INSERT INTO ways_nodes(id, node_id, position) VALUES (?,?,?);', to_db)



''' ways tags section'''

cur.execute('''DROP TABLE IF EXISTS ways_tags''') # exucutes an sql statement
conn.commit()


#id	key	value	type

cur.execute('''
    CREATE TABLE ways_tags(id INTEGER, key TEXT, value TEXT, type TEXT)
''')
# commit the changes
conn.commit()
# Read the csv file as a dictionary, 
# then format as a list of tuples


with open('ways_tags.csv', 'rb') as fin:
    dr = csv.DictReader(fin)
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]
    
cur.executemany('INSERT INTO ways_tags(id, key, value, type) VALUES (?,?,?,?);', to_db)



conn.commit()
conn.close


<function close>